In [1]:
class Unit:
    def __init__(self, faction, position, attack=3):
        self.faction = faction
        self.position = position
        self.hp = 200
        self.attack = attack

In [2]:
from operator import itemgetter
import heapq
import math

class Battle:
    def __init__(self, file, elf_attack = 3):
        self.battlefield = []
        self.units = {}
        self.rounds = 0
        for y, line in enumerate(file):
            row = []
            for x, char in enumerate(line.strip()):
                if char == '#':
                    row.append(False)
                else:
                    row.append(True)
                if char == 'E':
                    self.units[(y, x)] = Unit('E', (y, x), elf_attack)
                elif char == 'G':
                    self.units[(y, x)] = Unit('G', (y, x))
            self.battlefield.append(row)
        
    def __str__(self):
        string = ""
        for y, row in enumerate(self.battlefield):
            for x, cell in enumerate(row):
                if not cell:
                    string += '#'
                elif (y, x) in self.units:
                    string += self.units[(y, x)].faction
                else:
                    string += '.'
            string += "\n"
        return string.strip()
    
    def neighbors(self, cell):
        y, x = cell
        return [(y-1, x), (y+1, x), (y, x-1), (y, x+1)]
    
    def passable(self, cell):
        y, x = cell
        return self.battlefield[y][x] and (y, x) not in self.units
    
    def dijkstra(self, unit):
        # Contains the current distance and predecessor for each cell, even impassable ones
        paths = [[(math.inf, (0, 0)) for _ in range(len(self.battlefield[0]))] for _ in range(len(self.battlefield))]
        # Contains the cells taht recently got an update to their distance and predecessor, in order of distance.
        # It is initialized with the origin cell.
        queue = [(0, unit.position)]
        paths[unit.position[0]][unit.position[1]] = (0, unit.position)
        
        while queue:
            (d, (y, x)) = heapq.heappop(queue)
            if paths[y][x][0] < d:
                # Has already been updated with better results, we discard this one
                continue
            for (y2, x2) in self.neighbors((y, x)):
                if not self.passable((y2, x2)):
                    continue
                (d2, prev2) = paths[y2][x2]
                
                if (d + 1, (y, x)) < (d2, prev2):
                    # We choose a new predecessor if it is closer, or at same distance but in lower reading order
                    paths[y2][x2] = (d + 1, (y, x))
                    heapq.heappush(queue, (d+1, (y2, x2)))
        return paths

    def attack(self, unit):
        targets = [enemy for enemy in self.units.values() if enemy.faction != unit.faction and enemy.position in self.neighbors(unit.position)]
        if not targets:
            # No target in range
            return
        target = min(targets, key = lambda target: (target.hp, target.position))
        target.hp -= unit.attack
        if target.hp <= 0:
            # We remove the dead target
            self.units.pop(target.position)
                
    def move(self, unit):
        enemies = [enemy for enemy in self.units.values() if enemy.faction != unit.faction]
        if not enemies:
            # The battle is over
            return False
        paths = self.dijkstra(unit)
        in_reach = [(y, x) for enemy in enemies for (y, x) in self.neighbors(enemy.position) if paths[y][x][0] < math.inf]
        if not in_reach:
            # No cell in reach, the unit can not move
            return True
        
        # The cell to try and reach is chosen first by order of proximity, and then by reading order
        (d, (y, x)) = min((paths[y][x][0], (y, x)) for (y, x) in in_reach)
        y_prev, x_prev = paths[y][x][1]
        while (y_prev, x_prev) != unit.position:
            # We go back along the shortest path until the very first step to take
            y, x = y_prev, x_prev
            y_prev, x_prev = paths[y][x][1]
        
        # We make a move
        self.units.pop(unit.position)
        unit.position = (y, x)
        self.units[unit.position] = unit
        return True
    
    def turn(self, unit):
        if not self.move(unit):
            # No enemies, the battle is over
            return False
        self.attack(unit)
        return True
    
    def play_round(self):
        for pos in sorted(self.units.keys()):
            unit = self.units.get(pos)
            if not unit:
                # Probably dead
                continue
            if not self.turn(unit):
                # No enemies, the battle is over
                return False
        self.rounds += 1
        return True
    
    def play(self):
        # Play until the battle is over
        while self.play_round():
            pass
        
    def score(self):
        # Score at the end of the game
        self.play()
        return self.rounds * sum(unit.hp for unit in self.units.values())

In [3]:
# Input
with open("Input/15.txt") as file:
    battle = Battle(file)
    
print(battle)

################################
##########G###.#################
##########..G#G.G###############
##########G......#########...###
##########...##.##########...###
##########...##.#########G..####
###########G....######....######
#############...............####
#############...G..G.......#####
#############.............######
############.............E######
######....G..G.........E....####
####..G..G....#####.E.G.....####
#####...G...G#######........####
#####.......#########........###
####G.......#########.......####
####...#....#########.#.....####
####.#..#...#########E#..E#..###
####........#########..E.#######
###......#..G#######....########
###.......G...#####.....########
##........#............#########
#...##.....G......E....#########
#.#.###..#.....E.......###.#####
#######................###.#####
##########.......E.....###.#####
###########...##........#...####
###########..#####.............#
############..#####.....#......#
##########...######...........##
#########.

In [4]:
result = battle.score()
print("Part 1: {}".format(result))

Part 1: 229950


In [5]:
low = 3
high = 200

while low <= high:
    mid = (low + high) // 2
    with open("Input/15.txt") as file:
        battle = Battle(file, mid)
    nb_elves = len([unit for unit in battle.units.values() if unit.faction == 'E'])
    battle.play()
    nb_survivors = len([unit for unit in battle.units.values() if unit.faction == 'E'])
    if nb_survivors == nb_elves:
        # At least enough attack: search lower
        high = mid - 1
    else:
        # Not enough damage: search higher
        low = mid + 1

# At the end, we found the bound at low
with open("Input/15.txt") as file:
    battle = Battle(file, low)
print("Part 2: {}".format(battle.score()))

Part 2: 54360
